# Cloud Workshop Azure Databricks
## 10. Model Deployment Azure Container Instance (ACI) Azure Kubernetes Service (AKS)
<img src="https://raw.githubusercontent.com/retkowsky/images/master/AzureDatabricksLogo.jpg"><br>
V1.4 01/07/2000

# Documentation
Présentation https://azure.microsoft.com/fr-fr/services/databricks/

Documentation Azure Databricks : https://docs.microsoft.com/fr-fr/azure/databricks/

Documentation Azure ML : https://docs.microsoft.com/en-us/azure/machine-learning/

Github : https://github.com/Azure/MachineLearningNotebooks/tree/master/how-to-use-azureml/azure-databricks

## 0. Paramétrage

### 0.1 Azure Databricks cluster

In order to run this notebook properly, it must be attached to an Azure Databricks cluster that satisfies the following requirements:
  
- Use a DBR with Python 3, this notebook is demoed with DBR 6.5

### 0.2 Installer MLflow

Next, install the MLflow Python library using the following steps:

1. Create the library with the Source `Upload Python Egg or PyPI` and the versioned Pip library name:
  - `mlflow` - if you want to always use the latest, else use syntax `mlflow=1.7.0` to get a dedicated version of MLflow
  
2. Attach the library to the cluster.

### 0.3 Installer Azure ML MLflow SDK, ONNX

Once a cluster has been launched with the configuration described in **Launch an Azure Databricks cluster**, install the Azure Machine Learning SDK using the following steps:

1. Create the library with the Source ``Upload Python Egg or PyPI`` and the Pip library name:
  - `azureml-mlflow`, `skl2onnx`, `onnxruntime`
     
2. Attach the library to the cluster.

### 0.4 Azure ML Workspace

In [8]:
import datetime
now = datetime.datetime.now()
print(now)

2020-07-06 08:20:28.891130

In [9]:
import sys
sys.version

Out[1]: '3.7.3 (default, Mar 17 2020, 19:42:22) \n[GCC 5.4.0 20160609]'

In [10]:
import azureml
from azureml.core import Workspace

subscription_id = "AREMPLACER" #Votre ID Azure ML service
resource_group = "AzureML_DB-rg" # Le ressource groupe Azure ML service
workspace_name = "AzureML_DB" # Le nom Azure ML service
workspace_region = "west europe" # région Azure ML service

In [11]:
workspace = Workspace.create(name = workspace_name,
                             subscription_id = subscription_id,
                             resource_group = resource_group,
                             location = workspace_region,
                             exist_ok=True)

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code DLDZP6X2P to authenticate.
Interactive authentication successfully completed.

### 0.5 (Optional) MLflow tracking server

MLflow can collect data about a model training session, such as validation accuracy. It can also save artifacts produced during the training session, such as a PySpark pipeline model.

By default, these data and artifacts are stored on the cluster's local filesystem. However, they can also be stored remotely using an [MLflow Tracking Server](https://mlflow.org/docs/latest/tracking.html).

In [13]:
import mlflow
mlflow.__version__

# We are using the hosted mlflow tracking server

# If we want to use Azure ML MLflow tracking server, set the tracking URI
azureml_mlflow_uri = workspace.get_mlflow_tracking_uri()
mlflow.set_tracking_uri(azureml_mlflow_uri)

## 1. ML model

In [15]:
experiment_name = "Experience Wine dataset"
mlflow.set_experiment(experiment_name)

INFO: 'Experience Wine dataset' does not exist. Creating a new experiment

In [16]:
%sh wget https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv

--2020-07-03 14:13:46-- https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84199 (82K) [application/x-httpd-php]
Saving to: ‘winequality-red.csv’

 0K .......... .......... .......... .......... .......... 60% 179K 0s
 50K .......... .......... .......... .. 100% 60.9M=0.3s

2020-07-03 14:13:47 (293 KB/s) - ‘winequality-red.csv’ saved [84199/84199]

In [17]:
wine_data_path = "/databricks/driver/winequality-red.csv"

In [18]:
import os
import warnings
import sys

import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet

import mlflow
import mlflow.sklearn
import mlflow.onnx
import onnx
import skl2onnx

def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2


def train_model(wine_data_path, model_path, alpha, l1_ratio):
    warnings.filterwarnings("ignore")
    np.random.seed(40)

    # Read the wine-quality csv file (make sure you're running this from the root of MLflow!)
    data = pd.read_csv(wine_data_path, sep=None)

    # Split the data into training and test sets. (0.75, 0.25) split.
    train, test = train_test_split(data)

    # The predicted column is "quality" which is a scalar from [3, 9]
    train_x = train.drop(["quality"], axis=1)
    test_x = test.drop(["quality"], axis=1)
    train_y = train[["quality"]]
    test_y = test[["quality"]]

    # Start a new MLflow training run 
    with mlflow.start_run():
        # Fit the Scikit-learn ElasticNet model
        lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
        lr.fit(train_x, train_y)

        predicted_qualities = lr.predict(test_x)

        # Evaluate the performance of the model using several accuracy metrics
        (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

        print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)

        # Log model hyperparameters and performance metrics to the MLflow tracking server
        # (or to disk if no)
        mlflow.log_param("alpha", alpha)
        mlflow.log_param("l1_ratio", l1_ratio)
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("r2", r2)
        mlflow.log_metric("mae", mae)

        mlflow.sklearn.log_model(lr, model_path)
                
        initial_type = [('float_input', skl2onnx.common.data_types.FloatTensorType([None, test_x.shape[1]]))]
        onnx_model = skl2onnx.convert_sklearn(lr, initial_types=initial_type)
        print("onnx_model.type:", type(onnx_model))
        mlflow.onnx.log_model(onnx_model, "onnx-model")
        mlflow.set_tag("onnx_version", onnx.__version__)
        
        return mlflow.active_run().info.run_uuid

In [19]:
alpha_1 = 0.75
l1_ratio_1 = 0.25
model_path = 'model'
run_id1 = train_model(wine_data_path=wine_data_path, model_path=model_path, alpha=alpha_1, l1_ratio=l1_ratio_1)
model_uri = "runs:/"+run_id1+"/model"

Elasticnet model (alpha=0.750000, l1_ratio=0.250000):
 RMSE: 0.783730752565358
 MAE: 0.6165474987409882
 R2: 0.12970296126008685
onnx_model.type: <class 'onnx.onnx_ONNX_REL_1_7_ml_pb2.ModelProto'>

## 2. Image

In [21]:
import mlflow.azureml

model_image, azure_model = mlflow.azureml.build_image(model_uri=model_uri, 
                                                      workspace=workspace, 
                                                      model_name="wine-rating-model",
                                                      image_name="wine-model-container-image",
                                                      description="Sklearn ElasticNet image for rating wines", 
                                                      tags={
                                                        "alpha": str(alpha_1),
                                                        "l1_ratio": str(l1_ratio_1),
                                                      },
                                                      synchronous=True)

Registering model wine-rating-model
Creating image
Running...........................................................................
Succeeded
Image creation operation finished for image wine-model-container-image:3, operation "Succeeded"

> Prévoir 6 minutes de temps de traitements.

In [23]:
model_image.wait_for_creation(show_output=True)

Succeeded
Image creation operation finished for image wine-model-container-image:3, operation "Succeeded"

## 3. Déploiement du modèle dans Azure Container Instances (ACI)

## Déploiement du modèle de ML "dev" avec [Azure Container Instances (ACI)](https://docs.microsoft.com/en-us/azure/container-instances/)

The [ACI platform](https://docs.microsoft.com/en-us/azure/container-instances/) is the recommended environment for staging and developmental model deployments.

In [26]:
from azureml.core.webservice import AciWebservice, Webservice

dev_webservice_name = "wine-model-dev" # Nom unique
dev_webservice_deployment_config = AciWebservice.deploy_configuration()
dev_webservice = Webservice.deploy_from_image(name=dev_webservice_name, image=model_image, deployment_config=dev_webservice_deployment_config, workspace=workspace)

In [27]:
dev_webservice.wait_for_deployment()

ACI service creation operation finished, operation "Succeeded"

> Prévoir 5 minutes de temps de traitements.

## 4. Test du modèle de ML déployé dans ACI

In [30]:
import numpy as np
import pandas as pd
from sklearn import datasets

data = pd.read_csv(wine_data_path, sep=None)
train, _ = train_test_split(data)
train_x = train.drop(["quality"], axis=1)
sample = train_x.iloc[[0]]
query_input = list(sample.as_matrix().flatten())
sample_json = sample.to_json(orient="split")

In [31]:
import requests
import json

def query_endpoint_example(scoring_uri, inputs, service_key=None):
  headers = {
    "Content-Type": "application/json",
  }
  if service_key is not None:
    headers["Authorization"] = "Bearer {service_key}".format(service_key=service_key)
    
  print("Sending batch prediction request with inputs: {}".format(inputs))
  response = requests.post(scoring_uri, data=inputs, headers=headers)
  print("Response: {}".format(response.text))
  preds = json.loads(response.text)
  print("Received response: {}".format(preds))
  return preds

In [32]:
dev_scoring_uri = dev_webservice.scoring_uri

In [33]:
dev_prediction = query_endpoint_example(scoring_uri=dev_scoring_uri, inputs=sample_json)

Sending batch prediction request with inputs: {"columns":["fixed acidity","volatile acidity","citric acid","residual sugar","chlorides","free sulfur dioxide","total sulfur dioxide","density","pH","sulphates","alcohol"],"index":[129],"data":[[6.1,0.38,0.15,1.8,0.072,6.0,19.0,0.9955,3.42,0.57,9.4]]}
Response: [5.600460788314655]
Received response: [5.600460788314655]

## 5. Déploiement du modèle dans Azure Kubernetes Services (AKS)

### Option 1: Création nouveau cluster AKS

In [36]:
from azureml.core.compute import AksCompute, ComputeTarget

# Use the default configuration (you can also provide parameters to customize this)
prov_config = AksCompute.provisioning_configuration()

aks_cluster_name = "wine-prod-aks"

# Create the cluster
aks_target = ComputeTarget.create(workspace = workspace, 
                                  name = aks_cluster_name, 
                                  provisioning_configuration = prov_config)

# Wait for the create process to complete
aks_target.wait_for_completion(show_output = True)


Creating....................................................................................
SucceededProvisioning operation finished, operation "Succeeded"

> Prévoir 5 minutes de traitements

In [38]:
print("Provisionning :", aks_target.provisioning_state)
print("Erreur :", aks_target.provisioning_errors)

Provisionning : Succeeded
Erreur : None

### Option 2: Utilisation d'un cluster AKS existant

In [40]:
#from azureml.core.compute import AksCompute, ComputeTarget

# Get the resource id from https://porta..azure.com -> Find your resource group -> click on the Kubernetes service -> Properties
#resource_id = "/subscriptions/<your subscription id>/resourcegroups/<your resource group>/providers/Microsoft.ContainerService/managedClusters/<your aks service name>"


# Give the cluster a local name
#cluster_name = "<CLUSTER_NAME>"

# Attatch the cluster to your workgroup
#aks_target = AksCompute.attach(workspace=workspace, name=cluster_name, resource_id=resource_id)

#compute = ComputeTarget.attach(workspace, cluster_name, attach_config)

# Wait for the operation to complete
#compute.wait_for_completion(True)
#print(compute.provisioning_state)
#print(compute.provisioning_errors)

## Déploiement du modèle :

In [42]:
from azureml.core.webservice import Webservice, AksWebservice

# Set configuration and service name
prod_webservice_name = "wine-model-prod"

prod_webservice_deployment_config = AksWebservice.deploy_configuration()

# Deploy from image
prod_webservice = Webservice.deploy_from_image(workspace = workspace, 
                                               name = prod_webservice_name,
                                               image = model_image,
                                               deployment_config = prod_webservice_deployment_config,
                                               deployment_target = aks_target)

In [43]:
# Wait for the deployment to complete
prod_webservice.wait_for_deployment(show_output = True)

Running......
Succeeded
AKS service creation operation finished, operation "Succeeded"

## 6. Test du modèle déployé dans AKS

In [45]:
import numpy as np
import pandas as pd
from sklearn import datasets

data = pd.read_csv(wine_data_path, sep=None)
train, _ = train_test_split(data)
train_x = train.drop(["quality"], axis=1)
sample = train_x.iloc[[0]]
query_input = list(sample.as_matrix().flatten())
sample_json = sample.to_json(orient="split")

In [46]:
import requests
import json

def query_endpoint_example(scoring_uri, inputs, service_key=None):
  headers = {
    "Content-Type": "application/json",
  }
  if service_key is not None:
    headers["Authorization"] = "Bearer {service_key}".format(service_key=service_key)
    
  print("Sending batch prediction request with inputs: {}".format(inputs))
  response = requests.post(scoring_uri, data=inputs, headers=headers)
  preds = json.loads(response.text)
  print("Received response: {}".format(preds))
  return preds

In [47]:
prod_scoring_uri = prod_webservice.scoring_uri
prod_service_key = prod_webservice.get_keys()[0] if len(prod_webservice.get_keys()) > 0 else None

In [48]:
prod_prediction = query_endpoint_example(scoring_uri=prod_scoring_uri, service_key=prod_service_key, inputs=sample_json)

Sending batch prediction request with inputs: {"columns":["fixed acidity","volatile acidity","citric acid","residual sugar","chlorides","free sulfur dioxide","total sulfur dioxide","density","pH","sulphates","alcohol"],"index":[1296],"data":[[6.6,0.63,0.0,4.3,0.093,51.0,77.5,0.99558,3.2,0.45,9.5]]}
Received response: [5.562185017399966]

## 7. Mise à jour du modèle

### 7.1 Training d'un nouveau modèle de ML

In [51]:
alpha_2 = 0.5
l1_ratio_2 = 0.8
run_id2 = train_model(wine_data_path=wine_data_path, model_path=model_path, alpha=alpha_2, l1_ratio=l1_ratio_2)
model_uri = "runs:/"+run_id2+"/model"

Elasticnet model (alpha=0.500000, l1_ratio=0.800000):
 RMSE: 0.8320145539945119
 MAE: 0.6657081587004348
 R2: 0.019165855890777572
onnx_model.type: <class 'onnx.onnx_ONNX_REL_1_7_ml_pb2.ModelProto'>

In [52]:
# Création de l'image
import mlflow.azureml

model_image_updated, azure_model_updated = mlflow.azureml.build_image(model_uri=model_uri,
                                                                      workspace=workspace, 
                                                                      model_name="wine-rating-model",
                                                                      image_name="wine-model-container-image",
                                                                      description="Sklearn ElasticNet image for rating wines", 
                                                                      tags={
                                                                        "alpha": str(alpha_2),
                                                                        "l1_ratio": str(l1_ratio_2),
                                                                      },
                                                                      synchronous=False)

Registering model wine-rating-model
Creating image

In [53]:
model_image_updated.wait_for_creation(show_output=True)

Running.......................................................................
Succeeded
Image creation operation finished for image wine-model-container-image:4, operation "Succeeded"

> Prévoir 5 minutes de traitement

### 7.2 Déploiement du nouveau modèle dans AKS

In [56]:
prod_webservice.update(image=model_image_updated)

In [57]:
prod_webservice.wait_for_deployment(show_output = True)

Running............
Succeeded
AKS service creation operation finished, operation "Succeeded"

### 7.3 Test du nouveau modèle déployé AKS

In [59]:
prod_prediction_updated = query_endpoint_example(scoring_uri=prod_scoring_uri, service_key=prod_service_key, inputs=sample_json)

Sending batch prediction request with inputs: {"columns":["fixed acidity","volatile acidity","citric acid","residual sugar","chlorides","free sulfur dioxide","total sulfur dioxide","density","pH","sulphates","alcohol"],"index":[1296],"data":[[6.6,0.63,0.0,4.3,0.093,51.0,77.5,0.99558,3.2,0.45,9.5]]}
Received response: [5.556092568433957]

## 8. Suppression des ressources

In [61]:
dev_webservice.delete()
prod_webservice.delete()
aks_target.delete()